# Data Exploration for Historians


## A very gentle and practical introduction

In this lecture we continue exploring dataframes, but slowly dive into more analytical topics. We first show how to manipulate dataframes and try to understand and describe the properties of the data we're working with. Later on we move on the more statistical topics: hypothesis testing and regression.

This course doesn't provide a formal and rigorous introduction, but similarly to the Python part, we hope that by informally discussing and presenting more technical topics, historians are able to intuitively grasp how data analysis may be helpful for their research. We provde links to other handbooks and courses below.

## Age, Gender and Disability in London

For our first statistical exploration we use a small sample of (anonymised) census data from London. This records for each person in London their place, age, gender and disability. 

Gender, place are called variables. We first study them individually and later turn to analysing and interpreting their relationship.

# Opening and manipulating dataframes with Pandas

As in the previous notebook we first load all the tools and packages that we need for our analysis. The first line instructs the  environment to show all plots in the notebook. Line two and three import different libraries. In both cases we use an abbreviation to reference to these libraries later on (you'll notice 'pd' returning everywhere. This is just to save you time typing. In the last line we intruct Python to use `seaborn` for plotting (set it as default) this is just to make things look prettier! 

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()

Below we load the data. The syntax should be familiar from the previous notebook. Basically we use the function `read_csv` from the Pandas `pd` library to load a file on the location `'data/icem/EW01_london_sample.csv'`. The last argument `index_col` indicates the want the first colums (index 0) to serve as row index for our dataframe.

In [ ]:
df = pd.read_csv('data/icem/EW01_london_sample.csv',index_col=0)

In [ ]:
df = df.sample(frac=.5, random_state=0).reset_index()

Now we can print the first rows with `.head()`

In [ ]:
df.head()

Pandas provides a few methods for interpreting and understanding the data we are working with. Firstly we can print the `.shape` attribute (not a function, remember?) which express the basic structure of the dataframe.

In [ ]:
df.shape

Dataframes are structured data and consist of rows and columns. Row are usually observations or records, in this case individuals living in London. The columns are attributes or features of these records. In this case they capture specific attributes of people, the district where they live, their sex, age and disability.

To understand the size of the dataframe, `.shape` returns a tuple with the first element indicating the number of rows and the second the number of columns.

In data analysis dataframes are the most common object to work with. As we demonstrate below, pandas will provide you with many tools to understand and made sense of the this information. A useful methods that gives a general description of the content is `.info()`. Applying this method to `df` we get:

In [ ]:
df.info()

We observe the same numbers for rows and columns. What's interesting here is the description of the columns.

- `Column`: the column name
- `Non-Null Count`: often dataframes will contain missing data. This happens, for example when the attribute for a specific records is missing or unobserved. For example, we lack the sex of a specific individual. This columns tells us how complete the column is. Luckily in this we don't have missing data. For each colums the number of non-null counts equals the nubmber of rows `4421`. Even though missing data is an important topic, it is outside the scope of this tutorial.
- `Dtype`: tells us the data type of each column or variable. This will play an important role in the remainder of this course, as the type of the variable often determines the desing of our analysis and experiments. This dataframes contains two data types 'int64' and 'object'. `int64` are integers or counts. `Age` record number of years since birth as a number i.e. `34` etc. `object` is a vaguer category often indicates textual content.  **[complete description]**.

Pandas tries to determine the dtype of each columns when opening the file, but can get it wrong (from our perspective). For example `DisCode1` 

Sometimes we change variables for convenience. e.g. gender often converted to a categorical variable.

Techniques we will apply arer related to the data type of variable.

## Variable types

In general, we distinguish between following variable
    - numerical: 
        - continuous: probabilities or rations (e.g. np.float64
        - discrete: counts, number of words, age in year (np.int64)
    - categorical:
        - binary: sex np.object or np.int8 
        - ordinal: can be ordered but the step sizes are not fixed or easy to interpret. For example the likert scale, ratings. (np.object or np.int16)
        - nominal: distinct categories that can not be ordered. e.g. Industry. or Disability.
        

## Inspection of colums: plotting data distributions.

We can further inspect variables, especially the categorical one by using the `value_counts()` method to a column. This counts how often each value occurs and ranks the seperate categories by the frequency of their occurence. For example we can easily pull up the number of records by their sex and plot the results.

We firstly observe that the `Sex` variable contains three categories `F`, `M` and `U`, the latter meaning `Unknown`.

In [ ]:
df['Sex'].value_counts()

Notice that we can do the same using a slightly different syntax. In this case we don't use squary bracks but the column name follows as an attribute of the dataframe. This won't work if the column name contain space, special characters of names that are also used to define methods or attributes. 

In [ ]:
df.Sex.value_counts()

By applying `.plot()` we easily visualize the distribution of the categories in each variable. For example, the line of code below plots occurence of each label in the `Sex` columns. The argument `kind` specifies the type of plot. In this case used `barplot` in which each categorie is represented by one bar.

In [ ]:
df['Sex'].value_counts().plot(kind='bar')

For sure, we could also made a lineplot (changing `'bar'` to `'line'`). However, here we already observe how the data type of the variables influences our analysis. The lineplot is clearly not the most appropriate, not only lesss readable, is suggest a continuity that isn't there.

In [ ]:
df['Sex'].value_counts().plot(kind='line')

To plot the distibution of the numerical variables we need to follow a slightly different strategy. A common approach is to plot the histogram. This basically tells us how often each element in a numeric variable occurs (there is more to it, but we come to that in a second).

In [ ]:
df['Age'].plot(kind='hist')

The figure above gives an indication of the distribution of age of 19th century Londoners. If we have a closer look at the graph, you'll notice that age are grouped in seperate 'bins', i.e. each bar cover approximate ten years in age difference. Depending on your interest you can make the plot more or less granular. 

How to read this plot? Basically it tells us data we around 450000 observation from people aged between 0 and 10 etc. The largest group are between ca. 20 and 30 years old.

We can, for example show the same distribution dividing `Age` into 100 bins.

In [ ]:
df['Age'].plot(kind='hist',bins=100)

Now, that's a stange peak there in the around. Often it is more helpful to determine the binsize ourselves. We can do this by passing a list with determine the start and end of each bin. Here we try to plot the distribution of age with year as the basic unit.

In [ ]:
df['Age'].plot(kind='hist',bins=range(0,101,1))

Of course, looking at the distribution only gets you so far. We need to describe it more accurately. Here is wher measure of location and dispersion can help us.

# Location and Dispersion

While the previous section, helps us to understand the structure and content of our data to some extent, it all remains fairly superficial. It gives a good overview, but doesn't provide much of a grip. To dig deeper and start describing the data we often have to rely on different strategies to summarize information. Looking at all the data points (for example all the years in the age column), we'd like to know what age is typical for certain groups. To better understand and summarize distibutions, we discusss some 'statistics' to help us 


[informal definition of statistic]

Each of these statistics help us to estimate where values are located or what is the most typical value.

### range

Determining the range of values is often a good start, we first extract the most extreme values, to know where all other elements in a variable are located. In pandas we can apply the `.min()` and `.max()` methods to a columsn with numerical values and subtract them to obtain the range of ages of poeple in London

In [ ]:
max_age = df.Age.max()

In [ ]:
min_age = df.Age.min()

In [ ]:
range_age = max_age - min_age

In [ ]:
print('Age maximum', max_age)
print('Age minimum', min_age)
print("Age range", range_age )

The oldest person in London 1901 (in our sample at least) was 104 years, the youngest was obviously just born. Anyway, this tells us the all ages fall between these limits.

## mean

The most common statistic to summarize a numeric variable is the mean. For example below I records the number of cats I observed each day of the week. I collected these counts and saved them in the variable 'cats_observed`

In [ ]:
cats_observed = [1,4,5,4,6,4,2]

The mean then could be interpreted as number cats of I expect to observed daily. This statistic is obtained by summing up all the cats I observed by the number of days

In [ ]:


sum_obs = np.sum(cats_observed)
len_obs = len(cats_observed)
print('total cats observed',sum_obs)
print('number of days ',len_obs)
print('mean',round(sum_obs /len_obs,2))


In pandas we can easily compute the mean for a variable by applying the `.mean()` method.

In [ ]:
df.Age.mean()

The question remains: is the mean as good estimate of location, are most of values close to the mean? In the 

In [ ]:

obs =[1,4,5,4,6,4,222]
sum_obs = np.sum(obs)
len_obs = len(obs)
round(sum_obs /len_obs,2)


In [ ]:
from scipy.stats import trim_mean
trim_mean(df.Age,0.1)

is the mean a meaningful valuable measure of location?
trimmed mean a "good compromise"

robust measures
median
middle of sorted list of values
 only one value
    vs mean which uses all values

In [ ]:
obs = [1,4,5,4,6,4,2]
obs.sort()
print(obs)
obs[2]

In [ ]:
obs = [1,4,5,4,6,4,222]
obs.sort()
obs[2]

In [ ]:
df.Age.median()

The median is often called a more robust measure of location, i.e. it is not sensitive the particular outliers, which is a benefit. On the other hand, is attempts to the measure location based on just one number, i.e. the number in the middle of a sorted variable. The mean on the other hand is based on all values in a variables, and therefore, at least in smaller samples, more sensitive to outliers than the median.

A good option is looking at quantiles, which takes into accout the middle values but also other relevant, for example 25% and 75% of the data.

In [ ]:
df.Age.quantile([.25,.5,.75])

More values in the range of 0.5 (almost equally) but bigger steps later on 15 but 25 are between 40 and 105.
**rewrite** transition to estimating spread.

# Estimating spread

While the previous statistics aimed at capturing the most 'typical' values in a series, the ones we discuss below try to compute the spread around the expected values. The helps us

A mean is meaningless unless we assess the extent to which values deviate from the mean, can we expect them to generally appear close to the mean or not. Looking the dispersion or variability within on variable is critical, and we discuss this topic in more detail later.

The measures of spread calculate the deviation between our estimate of location (mean of median) and all other obeserved values. 

One of the most common measures of spread is the standard deviation.

To compute the standard deviations we first need to calculat the variance, which bacically tells us how distant is each observation is located from the mean. 

If we return to example with cats observed by day, we first compute the mean.

In [ ]:
cats_observed = [1,4,5,4,6,4,2]
mean = np.mean(cats_observed)
mean

Then we calculate the distance of each value to the mean and take the power of two of this value. This ensure that all values of positive. We are interested here in the distance and not in the direction.

In [ ]:
np.power(2,2)

In [ ]:
np.power(4,2)

In [ ]:
distances = []
for v in cats_observed:
    distances.append(np.power(v - mean,2))
distances

Then we sum all the distance and divide by the number of observation minus one. The reason for this isn't important to discuss here, it is related to the idea of degrees of freedom. However, a diversion would obscure more than explain. Also, we working with many obseration, it basically doesn't matter if you divide by `n` of `n - 1`. 

In [ ]:
sum_of_distances = np.sum(distances)

In [ ]:
variance = sum_of_distances / (len(distances) - 1)
variance

Then the standard deviation is the square root of this value.

In [ ]:
np.sqrt(variance)

Luckily, there exists, of course, and method in Pandas for compute the standard deviation (or variance if you like). We first convert the list to instance of type `pd.Series` and then apply the `.std()` method.

In [ ]:
co = pd.Series(cats_observed)
print('variance',co.var())
print('standard deviation',co.std())

Of course we can now also compute the standard deviation of age in London population.

In [ ]:
df.Age.std()

It is of course hard to pin down a meaning on this number. Is the standard deviation high? When does it matter?

The value of computing these statistics becomes more apparent when we start comparing groups and periods with each other. For example if we compare two means, the standard deviation helps is interpreting if the difference between means points to substantial divergence. We will come back to this later. 

Another common metric is the median absolute deviation from the median or simple MAD. This is similar to the standard deviation but we simple replace the mean with median.

In [ ]:
cats_observed.sort()
print(cats_observed)
median = cats_observed[int(len(cats_observed)/2)]
print(median)

In [ ]:
distances = []
for v in cats_observed:
    distances.append(np.abs(v - median))
distances

In [ ]:
distances.sort()
distances

In [ ]:
distances[3]

In [ ]:
from scipy.stats import median_abs_deviation
median_abs_deviation(cats_observed)

In [ ]:
median_abs_deviation(df.Age)

# Grouping and comparing data

The measures we discussed in the preceding section, are especially useful when it comes to comparing subgroups in our data. As Luke Blaxill explained in the first series of lectures, digital history often involves making meaningfull and intelligent comparisons for understanding historical patterns. 


This means being question driven and clear about the expectations. Of course, often the data will point in the opposite directions of our expectations but that's OK. It is through conversation with empericial materials—asking questions, analysing answers—that we gradually build new insights. 

Below we will have a closer look at the distribution of age in late Victorian London. From describing the data, we will attempt more and more understand and explain the variability in the data. 

## Sex and Age Differences
### Studying the relation between numerical and categorical variables

Are women, on average, older then men? If we plot the distribution of age with a histograms, you observe a slightly camel-like shape. Two peaks, one in the category of very young, between 0 and 10, and later, the distribution seems to peak around 25 year. Which is a kind-off strange pattern.

A first question we ask is relating age to gender and how do difference influences the shape of the overall distribution. 

In [ ]:
df.Age.plot(kind='hist',bins=range(100))

Pandas, fortunately, provides are very useful method to compare subgroups in the dataframe: `.groupby`. Below we use groupby to compare the average `Age` by `Sex`.

In [ ]:
df.groupby('Sex')['Age'].mean()

This syntax may come across as confusing at first sight, so let's have bit closer look at what's happening here. Groupby returns the mean age for each label in the `Sex`. It does so in three steps, which will replicate here to show you the process, but you can forget the syntax later. 

First we split the dataframe by label or key. These are unique values in the `Sex` variable.

In [ ]:
df.Sex.unique()

In [ ]:
df_f = df[df.Sex=='F']
df_m = df[df.Sex=='M']
df_u = df[df.Sex=='U']

This creates three seperate dataframes one for each lable 'F','M' and 'U'.

In [ ]:
df_f.shape,df_m.shape

Next we compute the mean for the "Age" column for each seperate dataframe.

In [ ]:
mean_f = df_f['Age'].mean()
mean_m = df_m['Age'].mean()
mean_u = df_u['Age'].mean()

print('mean age "F"',mean_f)
print('mean age "M"',mean_m)
print('mean age "U"',mean_u)


Lastly we combine these values in a `Series` object.

In [ ]:
mean_age_by_gender = pd.Series([mean_f,mean_m,mean_u],index=['F','M','U'],name='Age')
mean_age_by_gender

As you notice, the output of these steps is exactly the same as the one produced by applying `groupby`, it only required way more lines of code. It is often helpful to translate code in humane language: grouby the 'Age' variable by labels in the 'Sex' column and compute the mean for each category.

In [ ]:
df.groupby('Sex')['Age'].mean()

![groubpy](https://jakevdp.github.io/PythonDataScienceHandbook/figures/03.08-split-apply-combine.png)

Jake VanderPlas make a very apt visualisation of the process.

Of course, with groupby we can easily obtain other measures, for example compute the median for each 'Sex' category.

In [ ]:
df.groupby('Sex')['Age'].median()

While the median is generlly lower, the difference remain rather stable: whether we compute the mean or median, women in London seem generally one year older then men.

You can even compute multiple measures at once, for example the mean **and** the standard deviation. Below we apply `.agg()` to the `Age` columns. `.agg()` here takes a list of function that (`np.mean`, `np.std`] which are in turned applied to each subsection of the dataframe. The resulst is a 3x2 table, with the rows corresponding to the labels in `Sex` and the columns the different measures (`np.mean` and `np.std`)

In [ ]:
df.groupby('Sex')['Age'].agg([np.mean, np.std])

`.groupby` can also be used for plotting! Instead of return a number it will create a subplot (in this case histogram for each sex) and combine these into one figure. 

To make the figure more readable we added a few arguments
- `legend=True`: instructs Pandas to plot the label coresponding with each color in the plot
- `alpha=.5`: regulates transparency of the colours, which is always a good idea when combining histograms, otherwise some groups may become invisible.

In [ ]:
df[df.Sex.isin(['F','M'])].groupby('Sex')['Age'].plot(kind='hist',
                                                      legend=True,
                                                      bins=range(100),
                                                     alpha=.5)

Plotting the age distribution by gender helps us understanding the general camel-like pattern a bit better. In both categories, the histograms shows a concentration in the age category 20-30, but the trend is more pronounced among women than men.

But why at the organe bar systermically lower than the blue ones? Remember that the histogram

In [ ]:
df[df.Sex.isin(['F','M'])].groupby('Sex')['Age'].plot(kind='density',
                                                      legend=True,
                                                      alpha=.5)

Other ways of plotting an numerical variable and categorical one.

In [ ]:
sns.violinplot(x='Sex',y='Age',data= df[df.Sex.isin(['M','F'])])

In [ ]:
sns.boxplot(x='Sex',y='Age', data = df[df.Sex.isin(['M','F'])])

Visually, it looks like the the age for men and women in London didn't differ substantially. Later on we have a closer look if there exists a statistically 'significant' difference. 

But first, let's inspect the data in a it more detail. `.groubpy()` allows us to refine the comparison, for example, by inspecting age differences across gender and place. As the districts in London varied by their wealth and work patterns, we may expect the age to differ as well. 

We can simply add place 'RegDist' as a new level for grouping our the data. The code below, does the same as the `.groupby()` operation we used earlier, the only difference is that we split multiple time: first we split the data by place, and then, within each place we further split by gender and then compute the mean for Age, after which all the results are neatly grouped again.

In [ ]:
by_reg_gen = df.groupby(['RegDist','Sex'])['Age'].mean()
by_reg_gen

Working with output of this operation requires a bit more thought. The `.groupby` arranges data slightly differently depending if you group on one or more columns. This becomes apparent when printing the `type()` of the `.index` attribute.

In [ ]:
type(df.groupby('Sex')['Age'].median().index)

In [ ]:
type(by_reg_gen.index)

`by_reg_gen` is `MultiIndex` which is an important and useful different to be familiar with when working with dataframes.

In [ ]:
by_reg_gen['Bethnal Green']

In [ ]:
by_reg_gen['Bethnal Green':'Chelsea']

In [ ]:
by_reg_gen[:,'F']

In [ ]:
f_m_diff = by_reg_gen[:,'F'] - by_reg_gen[:,'M']
f_m_diff

## Hypothesis testing

At this stage, we can compute study and compare distribution of variables. But a question than immediately appears: are the differences we observe "significant", and what do we mean with "significance". In this section we have a closer look at hypothesis testing, but from a data driven perspective.

Traditional statistical methods, such as the Student's t-test arose in times of limited computing power and relied often on theorems and assumptions about the distributions and their properties. Explaining this requires many detour and implies a steep learning for the statistically unitiated. 

In this lecture therefore focus in more data-driven and hopefully intuitive procedures for signficance testing. We will rely on what is called the *permutation precudure*. The question we first address regards the relation between Sex and Age in Whitechapel and Westminster. We observed in the previous section that both districts deviate from the general pattern, namely a slightly older female population.

In [1]:
df_whitechapel = df[df.RegDist=='Stepney']
df_westminster = df[df.RegDist=='Westminster']
df_whitechapel.shape, df_westminster.shape

NameError: name 'df' is not defined

In [ ]:
df_whitechapel.groupby('Sex')['Age'].agg([np.mean,np.std])

In [2]:
df_westminster.groupby('Sex')['Age'].agg([np.mean,np.std])

NameError: name 'df_westminster' is not defined

In the case of Whitechapel, the difference between the mean is a 1.3 years. But is this a 'significant' difference. In the statistical language of hypothesis testing, if this difference a random artefact of the data, is the product of random chance? The latter will be our Null Hypothesis, namely that random chance explains the observed differences between the mean. The Alternate Hypothesis, is the "counterpoint" the Null Hypothesis, namely that the differences are not the result of random chance, there is a true difference. 

The procedure we will following is trying "prove" that the Alternative Hypothesis is True by rejecting the Null Hypothesis: we hope to show that if the Null Hypethesis is true, the observed 

In other words: the results derived from the conflict with the assumption that Null Hypothesis is. In this case will accept the Alternative Hypothesis.


Let's convert this to the case study at hand. 

What do we mean with random? Essentially we like to prove that differences in age between the sexes is related to 

Any subsection 

Directionality of the alternative hypothesis
different
greater than
smaller than


Central limit theorem


mean converges to a normal distribution

In [ ]:
mean_sampled = []
for _ in range(500):
    sample = df_whitechapel['Age'].sample(20)
    mean_sampled.append(sample.mean())
pd.Series(mean_sampled).plot(kind='density')
mean_sampled = []
for _ in range(5000):
    sample = df_whitechapel['Age'].sample(20)
    mean_sampled.append(sample.mean())
pd.Series(mean_sampled).plot(kind='density')

In [ ]:
df_wh_F = df_whitechapel[df_whitechapel.Sex=='F']
df_wh_M = df_whitechapel[df_whitechapel.Sex=='M']
df_wh_P = df_whitechapel[df_whitechapel.Sex.isin(['F','M'])]


In [ ]:
mean_sampled = []
for _ in range(1000):
    sample = df_wh_M['Age'].sample(20)
    mean_sampled.append(sample.mean())
ax = pd.Series(mean_sampled).plot(kind='density')
ax.axvline(x = df_wh_F['Age'].mean(), color='black', lw=2)

In [ ]:
df_wh_F.shape,df_wh_M.shape

In [ ]:
len(a_idx),len(b_idx), len(all_idx)

In [ ]:
permutations = []
num_F = df_wh_F.shape[0]
all_idx = set(df_wh_P.index)
for _ in range(5000):
    a_idx = set(df_wh_P.sample(num_F).index)
    b_idx = all_idx - a_idx
    permutations.append(df_wh_P.loc[a_idx]['Age'].mean() - df_wh_P.loc[b_idx]['Age'].mean())

In [ ]:
pd.Series(permutations).plot(kind='hist')

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(5, 5))
ax.hist(permutations, bins=11, rwidth=0.9)
ax.axvline(x = df_wh_F['Age'].mean() - df_wh_M['Age'].mean(), color='black', lw=2)
#ax.text(50, 190, 'Observed\ndifference')
ax.set_xlabel('Mean age differences')
ax.set_ylabel('Frequency')

In [ ]:
np.mean(np.array(permutations) <= df_wh_F['Age'].mean() - df_wh_M['Age'].mean())

In [ ]:
from scipy.stats import ttest_ind
ttest_ind(df_wh_F['Age'],df_wh_M['Age'], alternative='less')

## Gender and Disability
### Relations between categorical variables

In [ ]:
df['Disability'] = df['DisCode1'] > 0 

In [ ]:
p = df.groupby('Sex')['Disability'].agg([np.sum, len])

In [ ]:
p['no_disab']  = p['len'] - p['sum']

In [ ]:
p.T

In [ ]:
import random
def perm_fun(x, nA, nB):
    n = nA + nB
    idx_B = set(random.sample(range(n), nB))
    idx_A = set(range(n)) - idx_B
    return x.loc[idx_B].mean() - x.loc[idx_A].mean()


In [ ]:
obs_pct_diff = 100 * ((4772 / 1151473) - (4412 / 977986))
print(f'Observed difference: {obs_pct_diff:.4f}%')
conversion = [0] * (1146701+973574)
conversion.extend([1] * (4772+4412))
conversion = pd.Series(conversion)
perm_diffs = [100 * perm_fun(conversion, 1151473, 977986) for _ in range(100)]
fig, ax = plt.subplots(figsize=(5, 5))
ax.hist(perm_diffs, bins=11, rwidth=0.9)
ax.axvline(x=obs_pct_diff, color='black', lw=2)
#ax.text(0.06, 200, 'Observed\ndifference', bbox={'facecolor':'white'})
ax.set_xlabel('Conversion rate (percent)')
ax.set_ylabel('Frequency')

In [ ]:
from scipy import stats
survivors = np.array([[4772, 1146701], [4412, 973574]])
chi2, p_value, df, _ = stats.chi2_contingency(survivors)
print(f'p-value : {p_value*2 / 2:.8f}')
chi2